In [52]:
import numpy as np
import re
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors
from sklearn.decomposition import PCA

In [3]:
df_23 = pd.read_csv('./Dataset/FIFA23_official_data.csv')

df_22 = pd.read_csv('./Dataset/FIFA22_official_data.csv')


In [4]:
df_22

,ID,Name,Age,Photo,Nationality,Flag,Overall,Potential,Club,Club Logo,...,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Best Position,Best Overall Rating,Release Clause,DefensiveAwareness
0,212198,Bruno Fernandes,26,https://cdn.sofifa.com/players/212/198/22_60.png,Portugal,https://cdn.sofifa.com/flags/pt.png,88,89,Manchester United,https://cdn.sofifa.com/teams/11/30.png,...,65.0,12.0,14.0,15.0,8.0,14.0,CAM,88.0,€206.9M,72.0
1,209658,L. Goretzka,26,https://cdn.sofifa.com/players/209/658/22_60.png,Germany,https://cdn.sofifa.com/flags/de.png,87,88,FC Bayern München,https://cdn.sofifa.com/teams/21/30.png,...,77.0,13.0,8.0,15.0,11.0,9.0,CM,87.0,€160.4M,74.0
2,176580,L. Suárez,34,https://cdn.sofifa.com/players/176/580/22_60.png,Uruguay,https://cdn.sofifa.com/flags/uy.png,88,88,Atlético de Madrid,https://cdn.sofifa.com/teams/240/30.png,...,38.0,27.0,25.0,31.0,33.0,37.0,ST,88.0,€91.2M,42.0
3,192985,K. De Bruyne,30,https://cdn.sofifa.com/players/192/985/22_60.png,Belgium,https://cdn.sofifa.com/flags/be.png,91,91,Manchester City,https://cdn.sofifa.com/teams/10/30.png,...,53.0,15.0,13.0,5.0,10.0,13.0,CM,91.0,€232.2M,68.0
4,224334,M. Acuña,29,https://cdn.sofifa.com/players/224/334/22_60.png,Argentina,https://cdn.sofifa.com/flags/ar.png,84,84,Sevilla FC,https://cdn.sofifa.com/teams/481/30.png,...,82.0,8.0,14.0,13.0,13.0,14.0,LB,84.0,€77.7M,80.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16705,240558,18 L. Clayton,17,https://cdn.sofifa.com/players/240/558/18_60.png,England,https://cdn.sofifa.com/flags/gb-eng.png,53,70,Cheltenham Town,https://cdn.sofifa.com/teams/1936/30.png,...,12.0,55.0,54.0,52.0,50.0,59.0,GK,52.0,€238K,NaN
16706,262846,�. Dobre,20,https://cdn.sofifa.com/players/262/846/22_60.png,Romania,https://cdn.sofifa.com/flags/ro.png,53,63,FC Academica Clinceni,https://cdn.sofifa.com/teams/113391/30.png,...,12.0,57.0,52.0,53.0,48.0,58.0,GK,53.0,€279K,5.0
16707,241317,21 Xue Qinghao,19,https://cdn.sofifa.com/players/241/317/21_60.png,China PR,https://cdn.sofifa.com/flags/cn.png,47,60,Shanghai Shenhua FC,https://cdn.sofifa.com/teams/110955/30.png,...,9.0,49.0,48.0,45.0,38.0,52.0,GK,47.0,€223K,21.0
16708,259646,A. Shaikh,18,https://cdn.sofifa.com/players/259/646/22_60.png,India,https://cdn.sofifa.com/flags/in.png,47,67,ATK Mohun Bagan FC,https://cdn.sofifa.com/teams/113146/30.png,...,13.0,49.0,41.0,39.0,45.0,49.0,GK,47.0,€259K,7.0


In [5]:
df_23

,ID,Name,Age,Photo,Nationality,Flag,Overall,Potential,Club,Club Logo,...,Real Face,Position,Joined,Loaned From,Contract Valid Until,Height,Weight,Release Clause,Kit Number,Best Overall Rating
0,209658,L. Goretzka,27,https://cdn.sofifa.net/players/209/658/23_60.png,Germany,https://cdn.sofifa.net/flags/de.png,87,88,FC Bayern München,https://cdn.sofifa.net/teams/21/30.png,...,Yes,"<span class=""pos pos28"">SUB","Jul 1, 2018",NaN,2026,189cm,82kg,€157M,8.0,NaN
1,212198,Bruno Fernandes,27,https://cdn.sofifa.net/players/212/198/23_60.png,Portugal,https://cdn.sofifa.net/flags/pt.png,86,87,Manchester United,https://cdn.sofifa.net/teams/11/30.png,...,Yes,"<span class=""pos pos15"">LCM","Jan 30, 2020",NaN,2026,179cm,69kg,€155M,8.0,NaN
2,224334,M. Acuña,30,https://cdn.sofifa.net/players/224/334/23_60.png,Argentina,https://cdn.sofifa.net/flags/ar.png,85,85,Sevilla FC,https://cdn.sofifa.net/teams/481/30.png,...,No,"<span class=""pos pos7"">LB","Sep 14, 2020",NaN,2024,172cm,69kg,€97.7M,19.0,NaN
3,192985,K. De Bruyne,31,https://cdn.sofifa.net/players/192/985/23_60.png,Belgium,https://cdn.sofifa.net/flags/be.png,91,91,Manchester City,https://cdn.sofifa.net/teams/10/30.png,...,Yes,"<span class=""pos pos13"">RCM","Aug 30, 2015",NaN,2025,181cm,70kg,€198.9M,17.0,NaN
4,224232,N. Barella,25,https://cdn.sofifa.net/players/224/232/23_60.png,Italy,https://cdn.sofifa.net/flags/it.png,86,89,Inter,https://cdn.sofifa.net/teams/44/30.png,...,Yes,"<span class=""pos pos13"">RCM","Sep 1, 2020",NaN,2026,172cm,68kg,€154.4M,23.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17655,269526,Deng Xiongtao,19,https://cdn.sofifa.net/players/269/526/23_60.png,China PR,https://cdn.sofifa.net/flags/cn.png,48,61,Meizhou Hakka,https://cdn.sofifa.net/teams/114628/30.png,...,No,"<span class=""pos pos29"">RES","Apr 11, 2022",NaN,2027,190cm,78kg,€218K,35.0,NaN
17656,267946,22 Lim Jun Sub,17,https://cdn.sofifa.net/players/267/946/22_60.png,Korea Republic,https://cdn.sofifa.net/flags/kr.png,48,64,Jeju United FC,https://cdn.sofifa.net/teams/1478/30.png,...,No,"<span class=""pos pos29"">RES","Jan 1, 2022",NaN,2026,195cm,84kg,€188K,21.0,NaN
17657,270567,A. Demir,25,https://cdn.sofifa.net/players/270/567/23_60.png,Turkey,https://cdn.sofifa.net/flags/tr.png,51,56,Ümraniyespor,https://cdn.sofifa.net/teams/113796/30.png,...,No,"<span class=""pos pos29"">RES","Jun 6, 2021",NaN,2023,190cm,82kg,€142K,12.0,NaN
17658,256624,21 S. Czajor,18,https://cdn.sofifa.net/players/256/624/21_60.png,Poland,https://cdn.sofifa.net/flags/pl.png,50,65,Fleetwood Town,https://cdn.sofifa.net/teams/112260/30.png,...,No,"<span class=""pos pos29"">RES","Jan 1, 2020",NaN,2021,187cm,79kg,€214K,40.0,NaN


In [72]:
# Filter df_22 and df_23 to keep only the common club names
common_clubs = set(df_22['Club']).intersection(set(df_23['Club']))
df_22_common = df_22[df_22['Club'].isin(common_clubs)]
df_23_common = df_23[df_23['Club'].isin(common_clubs)]

# Merge the filtered DataFrames
merged = pd.merge(df_22_common, df_23_common, on='ID', how='inner', suffixes=('_22', '_23'))

In [73]:
merged

,ID,Name_22,Age_22,Nationality_22,Overall_22,Potential_22,Club_22,Value_22,Weak Foot_22,Skill Moves_22,...,Real Face,Position_23,Joined,Loaned From,Contract Valid Until,Height_23,Weight_23,Release Clause_23,Kit Number,Best Overall Rating
0,212198,Bruno Fernandes,26,Portugal,88,89,Manchester United,€107.5M,3.0,4.0,...,Yes,"<span class=""pos pos15"">LCM","Jan 30, 2020",NaN,2026,179cm,69kg,€155M,8.0,NaN
1,209658,L. Goretzka,26,Germany,87,88,FC Bayern München,€93M,4.0,3.0,...,Yes,"<span class=""pos pos28"">SUB","Jul 1, 2018",NaN,2026,189cm,82kg,€157M,8.0,NaN
2,176580,L. Suárez,34,Uruguay,88,88,Atlético de Madrid,€44.5M,4.0,3.0,...,Yes,"<span class=""pos pos26"">LS","Jul 28, 2022",NaN,2022,182cm,86kg,€39.6M,9.0,NaN
3,192985,K. De Bruyne,30,Belgium,91,91,Manchester City,€125.5M,5.0,4.0,...,Yes,"<span class=""pos pos13"">RCM","Aug 30, 2015",NaN,2025,181cm,70kg,€198.9M,17.0,NaN
4,224334,M. Acuña,29,Argentina,84,84,Sevilla FC,€37M,3.0,4.0,...,No,"<span class=""pos pos7"">LB","Sep 14, 2020",NaN,2024,172cm,69kg,€97.7M,19.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7567,264057,J. Nieuwenhuizen,17,Australia,51,74,Melbourne City FC,€170K,1.0,1.0,...,No,"<span class=""pos pos28"">SUB","Apr 25, 2021",NaN,2024,184cm,74kg,€188K,40.0,NaN
7568,262373,F. Chalupniczak,20,Poland,51,63,Sutton United,€120K,2.0,1.0,...,No,"<span class=""pos pos29"">RES","Jul 1, 2021",NaN,2023,185cm,85kg,€254K,30.0,NaN
7569,263373,J. Searle,20,New Zealand,51,64,Swansea City,€120K,3.0,1.0,...,No,"<span class=""pos pos29"">RES","Jul 1, 2022",NaN,2024,199cm,85kg,€309K,20.0,NaN
7570,259718,F. Gebhardt,19,Germany,52,66,FC Basel 1893,€170K,3.0,1.0,...,No,"<span class=""pos pos28"">SUB",NaN,"<a href=""/team/896/fc-basel-1893/"">FC Basel 18...","Jun 30, 2023",188cm,83kg,NaN,23.0,NaN


In [74]:
transfers = merged[merged['Club_22'] != merged['Club_23']][['ID', 'Name_22','Club_22','Club_23']]

In [75]:
transfers

,ID,Name_22,Club_22,Club_23
2,176580,L. Suárez,Atlético de Madrid,Club Nacional de Football
7,181291,G. Wijnaldum,Paris Saint-Germain,Roma
16,212462,Alex Telles,Manchester United,Sevilla FC
22,200458,L. Digne,Everton,Aston Villa
27,195864,P. Pogba,Manchester United,Juventus
...,...,...,...,...
7538,254122,H. Cartwright,Hull City,Peterborough United
7541,258928,V. Sinisalo,Aston Villa,Burton Albion
7563,257138,B. Nna Noukeu,Crawley Town,Stoke City
7569,263373,J. Searle,Swansea City,Barnsley


In [77]:
transfers.to_csv('./rec_extended_data/ground_truth.csv')



## Approach 1: KNN

In [29]:
Unimportant_features  = ["Photo", "Flag", "Club Logo", "Wage", "Special", "International Reputation", "Work Rate", "Body Type", "Real Face", "Jersey Number", "Contract Valid Until", "Best Overall Rating","Joined","Loaned From"]
df_22.drop(Unimportant_features, axis = 1, inplace = True)


In [32]:
#One - Hot Encoding the Preferred Foot
df_22 = pd.get_dummies(df_22,columns = ["Preferred Foot"])

In [36]:
detailed_features = ['Age','Overall', 'Potential', 'Weak Foot', 'Skill Moves', 'Height', 'Weight', 'Crossing', 'Finishing',
       'HeadingAccuracy', 'ShortPassing', 'Volleys', 'Dribbling', 'Curve',
       'FKAccuracy', 'LongPassing', 'BallControl', 'Acceleration',
       'SprintSpeed', 'Agility', 'Reactions', 'Balance', 'ShotPower',
       'Jumping', 'Stamina', 'Strength', 'LongShots', 'Aggression',
       'Interceptions', 'Positioning', 'Vision', 'Penalties', 'Composure',
       'Marking', 'StandingTackle', 'SlidingTackle', 'GKDiving', 'GKHandling',
       'GKKicking', 'GKPositioning', 'GKReflexes', 'Preferred Foot_Left', 'Preferred Foot_Right']

In [37]:
df_22[detailed_features] = df_22[detailed_features].fillna(0)

In [41]:
# Clean height column
feet_inches_re = re.compile(r"(\d)'(\d+)")

# Define a function to convert feet'inches to cm
def feet_inches_to_cm(s):
    match = feet_inches_re.match(s)
    if match:
        feet = int(match.group(1))
        inches = int(match.group(2))
        return round((feet*12 + inches) * 2.54, 2)
    else:
        return float(s.replace('cm', ''))
df_22['Height'] = pd.to_numeric(df_22['Height'].apply(feet_inches_to_cm))

In [45]:
# Clean Weight Column
def convert_weight(weight_str):
    if 'kg' in weight_str:
        # If weight is in kg, remove the 'kg' suffix and return as float
        return float(weight_str.replace('kg', ''))
    elif 'lbs' in weight_str:
        # If weight is in lbs, convert to kg and return as float
        return float(weight_str.replace('lbs', '')) * 0.453592
    else:
        print("oops")
        # Return NaN if weight format is not recognized
        return pd.np.nan

# Apply the function to the Weight column and create a new column 'Weight_kg'
df_22['Weight'] = df_22['Weight'].apply(convert_weight)

In [60]:
df = df_22.select_dtypes(include = 'number')
df.shape

(16710, 45)

In [61]:
# Compute the correlation matrix
corr = df.corr()

In [62]:
df = df.fillna(0)

In [83]:
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler

# Separate the ID column and scale the data
X = df.drop('ID', axis=1).fillna(0)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Create the KNN model
k = 5 # number of neighbors to consider
knn = NearestNeighbors(n_neighbors=k+1) # add 1 to include the input player in the neighbors
knn.fit(X_scaled)

# Define a function to get the top 5 closest neighbors of a player
def get_top_5_neighbors(player_id):
    player_row = df[df['ID'] == player_id].drop('ID', axis=1).fillna(0)
    player_row_scaled = scaler.transform(player_row)
    distances, indices = knn.kneighbors(player_row_scaled)
    neighbor_ids = df.iloc[indices[0]][1:]['ID'].values # exclude the first index which is the input player
    return df[df['ID'].isin(neighbor_ids)].reset_index(drop=True)

In [84]:
data

,ID,Age,Overall,Potential,Weak Foot,Skill Moves,Height,Weight,Crossing,Finishing,...,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,DefensiveAwareness,Preferred Foot_Left,Preferred Foot_Right
0,212198,26,88,89,3.0,4.0,179.0,69.0,87.0,83.0,...,73.0,65.0,12.0,14.0,15.0,8.0,14.0,72.0,0,1
1,209658,26,87,88,4.0,3.0,189.0,82.0,75.0,82.0,...,85.0,77.0,13.0,8.0,15.0,11.0,9.0,74.0,0,1
2,176580,34,88,88,4.0,3.0,182.0,83.0,80.0,93.0,...,45.0,38.0,27.0,25.0,31.0,33.0,37.0,42.0,0,1
3,192985,30,91,91,5.0,4.0,181.0,70.0,94.0,82.0,...,65.0,53.0,15.0,13.0,5.0,10.0,13.0,68.0,0,1
4,224334,29,84,84,3.0,4.0,172.0,69.0,87.0,66.0,...,84.0,82.0,8.0,14.0,13.0,13.0,14.0,80.0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16705,240558,17,53,70,2.0,1.0,188.0,80.0,12.0,5.0,...,11.0,12.0,55.0,54.0,52.0,50.0,59.0,0.0,0,1
16706,262846,20,53,63,2.0,1.0,194.0,86.0,13.0,7.0,...,13.0,12.0,57.0,52.0,53.0,48.0,58.0,5.0,0,1
16707,241317,19,47,60,2.0,1.0,180.0,71.0,12.0,9.0,...,11.0,9.0,49.0,48.0,45.0,38.0,52.0,21.0,0,1
16708,259646,18,47,67,3.0,1.0,186.0,74.0,13.0,8.0,...,14.0,13.0,49.0,41.0,39.0,45.0,49.0,7.0,0,1


In [85]:
df_22[df_22["ID"]==212616	]

,ID,Name,Age,Nationality,Overall,Potential,Club,Value,Weak Foot,Skill Moves,...,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Best Position,Release Clause,DefensiveAwareness,Preferred Foot_Left,Preferred Foot_Right
70,212616,R. De Paul,27,Argentina,82,83,Atlético de Madrid,€35M,3.0,4.0,...,13.0,12.0,10.0,7.0,14.0,CM,€74.4M,68.0,0,1


In [86]:
player_id = 212198 # replace with the ID of the player you want to find the closest neighbors of
top_5_neighbors = get_top_5_neighbors(player_id)


In [87]:
top_5_neighbors

,ID,Age,Overall,Potential,Weak Foot,Skill Moves,Height,Weight,Crossing,Finishing,...,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,DefensiveAwareness,Preferred Foot_Left,Preferred Foot_Right
0,192985,30,91,91,5.0,4.0,181.0,70.0,94.0,82.0,...,65.0,53.0,15.0,13.0,5.0,10.0,13.0,68.0,0,1
1,204923,27,84,85,4.0,3.0,178.0,76.0,74.0,81.0,...,70.0,59.0,11.0,13.0,14.0,15.0,13.0,71.0,0,1
2,186942,30,85,85,4.0,4.0,180.0,80.0,78.0,80.0,...,75.0,67.0,11.0,14.0,5.0,5.0,13.0,76.0,0,1
3,212616,27,82,83,3.0,4.0,180.0,70.0,69.0,76.0,...,79.0,69.0,13.0,12.0,10.0,7.0,14.0,68.0,0,1
4,199823,28,82,82,3.0,4.0,179.0,70.0,81.0,73.0,...,75.0,62.0,8.0,11.0,6.0,7.0,10.0,72.0,0,1


In [98]:
ID=top_5_neighbors["ID"].tolist()
type(ID)
ID

[192985, 204923, 186942, 212616, 199823]

In [102]:
for id in ID:
    print(id)
    name = df_22[df_22["ID"]==id]
    print(name["Name"])

192985
3    K. De Bruyne
Name: Name, dtype: object
204923
18    M. Sabitzer
Name: Name, dtype: object
186942
47    I. Gündoğan
Name: Name, dtype: object
212616
70    R. De Paul
Name: Name, dtype: object
199823
158    Campaña
Name: Name, dtype: object
